In [9]:
import pandas as pd
import networkx as nx

from collections import OrderedDict
from tqdm import tqdm

## Read network file

In [10]:
df = pd.read_csv('rt-pol.txt')
df.head()

,target,source,timestamp
0,12464,7349,1286901355
1,6820,12569,1287988962
2,4336,15967,1286535938
3,16397,5927,1285134188
4,8829,13851,1285493596


## Generate graph

In [11]:
def generate_graph(df):
    """
    Generate two directed graph
    :param df: A df with source, target, timestamp columns,
               the tuple source, target is not unique, so for each
               occurrence we update the weight attribute
    :return: The main graph (G1), a the reverse one (G2)
    
    """

    # Create first directed graph
    G1 = nx.DiGraph()
    for row in df.itertuples():
        try:
            weight = G1[row.source][row.target]['weight']
            G1.add_edge(row.source, row.target, weight=weight+1)
        except KeyError:
            G1.add_edge(row.source, row.target, weight=1)

    # Create second directed graph
    G2 = nx.DiGraph()
    for row in df.itertuples():
        try:
            weight = G2[row.target][row.source]['weight']
            G2.add_edge(row.target, row.source, weight=weight+1)
        except KeyError:
            G2.add_edge(row.target, row.source, weight=1)

    return G1, G2

## Plurality Attr

In [12]:
def add_plurality_attribute(G1, G2, nodes):
    """
    Add plurality attribute.
    """
    for n in nodes:
        
        if(G1.degree(n) == 0):
            plurality = sys.maxsize
        else:
            sum_weight = 0
            for g2_node_source in G2.neighbors(n):
                sum_weight = sum_weight + G1.get_edge_data(g2_node_source, n)['weight']

            plurality = int(sum_weight/2) + 1
            
        G1.node[n]['plurality'] = plurality

    return G1

In [13]:
g1, g2 = generate_graph(df)

In [14]:
unique_nodes = set()
unique_nodes.update(df['source'].unique())
unique_nodes.update(df['target'].unique())
len(unique_nodes)

18470

In [15]:
g1 = add_plurality_attribute(g1, g2, unique_nodes)

In [49]:
def get_neighborhood(node, g1, g2, total_nodes):
    """
    Calculate nodes according at the neighborhood level.
    Using a node as bootstrap:
    - The first level is only the node, 
    - The second level is the first node and their neighbors
    - The third level .... 
    
    :param node: The boostrap node identifier
    :param g1: Main graph
    :param g2: Reverse graph
    :param total_nodes: Number of unique nodes in the main graph
    :return: A list with the initial node, the neighborhood level,
             and their nodes
    """
    old_nodes = set()
    new_nodes = set()
    
    neighborhood = []
    neighborhood.append({'inital_node':node,
                         's': 0, 
                         'nodes':[node]})

    new_nodes.add(node)
    last_nodes = None

    for idx, n in enumerate(range(total_nodes)):

        nodes_to_add = set()
        for new_node in new_nodes:
            nodes_to_add.update(g1.neighbors(new_node))
            nodes_to_add.update(g2.neighbors(new_node))

        old_nodes= old_nodes.union(new_nodes)

        new_nodes.clear()
        new_nodes.update(nodes_to_add)


        nodes_s_level = set(old_nodes).union(new_nodes)

        neighborhood.append({'inital_node':node,
                             's':idx + 1, 
                             'nodes':nodes_s_level})

        if len(nodes_s_level) == total_nodes:
            return neighborhood

In [119]:
def linear_threshold_rank(node, G1, G2, total_nodes):
    """
    Calculate the lTR for each node 
    - https://www.sciencedirect.com/science/article/pii/S0950705117304975
    """
    linear_threshold_rank = []
    
    PRINT_STEPS = False

    if PRINT_STEPS:
        print('-------------- Node: {} --------------'.format(node))
              
    neighborhoods = get_neighborhood(node, g1, g2, total_nodes)
    
    for neighborhood in neighborhoods:

        first_step = True

        nodes_to_add_group = set()

        # Get first neighbors (bootstrap nodes)
        neighbors = set()
        
        for bootstrap_node in neighborhood['nodes']:
            neighbors.add(bootstrap_node)
            neighbors.update(G1.neighbors(bootstrap_node))
            neighbors.update(G2.neighbors(bootstrap_node))

        group = set()
        group.update(neighbors)

        if PRINT_STEPS:
            print('I: 0 ; Neighbors: {0} ; count: {1}'.format(neighbors, len(neighbors)))

        depth_level = 0

        while( first_step or (len(nodes_to_add_group) >= 1) ):

            first_step = False

            neighbors.update(nodes_to_add_group)
            group.update(nodes_to_add_group)

            nodes_to_add_group.clear()

            if PRINT_STEPS:
                print('\t group: {1}'.format(n, group))

            next_nodes = set()
            for next_node in neighbors:
                next_nodes.update(G1.neighbors(next_node))

            # Nodes that can be influenced
            dispersion = set()
            dispersion = next_nodes - group

            if PRINT_STEPS:
                print('\t dispersion {0} '.format(dispersion))

            # For each dispersion node
            for n_sub_level in dispersion:
                plurality = G1.node[n_sub_level]['plurality']

                if PRINT_STEPS:
                    print('\t \t Reach node {0} | plurality {1}'.format(n_sub_level, plurality))

                group_influce = 0
                for node_group in group:
                    if(G1.get_edge_data(node_group, n_sub_level)):
                        group_influce = group_influce + G1.get_edge_data(node_group,n_sub_level)['weight']

                if PRINT_STEPS:
                    print('\t \t group {0} | influce {1}'.format(group, group_influce))

                if(group_influce >= plurality):
                    nodes_to_add_group.add(n_sub_level)

            if PRINT_STEPS:
                print('\t \t \t new group {0} '.format(nodes_to_add_group))
                print('{0} ; {1} ; {2}'.format(n, depth_level, len(group)))
                print()
                
                
            linear_threshold_rank.append({'node':node,
                                          'k':depth_level,
                                          'group_influenced':group.copy(),
                                          'number_influenced':len(group),
                                          's':neighborhood['s']})
            
            depth_level = depth_level +1 
        
    return linear_threshold_rank

In [120]:
%%time
results = linear_threshold_rank(1, g1, g2, 18470)

CPU times: user 2min 1s, sys: 44 ms, total: 2min 1s
Wall time: 2min 2s


In [121]:
df_results = pd.DataFrame(results)
df_results[['node','s','k','number_influenced','group_influenced']].sort_values(by=['node','s','k'])

,node,s,k,number_influenced,group_influenced
0,1,0,0,4,"{1, 5410, 5822, 15743}"
1,1,0,1,6,"{5792, 1, 5410, 14414, 5822, 15743}"
2,1,1,0,38,"{1, 4098, 5770, 1421, 16660, 11674, 7578, 5792..."
3,1,1,1,82,"{1, 4098, 7426, 16389, 9230, 6927, 16660, 9748..."
4,1,1,2,91,"{1, 4098, 7426, 5378, 16389, 9230, 6927, 16660..."
5,1,2,0,1298,"{1, 4098, 16387, 16389, 12297, 16397, 8213, 16..."
6,1,2,1,2404,"{8192, 1, 16387, 16389, 8197, 16397, 8207, 15,..."
7,1,2,2,2615,"{8192, 1, 16387, 16389, 8197, 16397, 8207, 15,..."
8,1,2,3,2660,"{8192, 1, 16387, 16389, 8197, 16397, 8207, 15,..."
9,1,2,4,2673,"{8192, 1, 16387, 16389, 8197, 16397, 8207, 15,..."
